# CNN model

In [33]:
import numpy as np
import pandas as pd
import os

def get_n_element_slices(dataframe: pd.DataFrame, n: int) -> list[pd.DataFrame]:
    if not isinstance(dataframe, pd.DataFrame):
        raise TypeError("Input must be a pandas DataFrame.")
    if not isinstance(n, int) or n <= 0:
        raise ValueError("n must be a positive integer.")
    if n > len(dataframe):
        return [] # No possible slices of length n if n is larger than the DataFrame

    slices = []
    for i in range(len(dataframe) - n + 1):
        slice_df = dataframe.iloc[i : i + n].copy()
        regularize_timestamp(slice_df)
        slices.append(slice_df)
    return slices

def regularize_timestamp(df: pd.DataFrame):
    start = df.iloc[0, 0]
    df["timestamp"] = df["timestamp"] - start


all_slices = []

data_path = "../data/clean"
for filename in os.listdir(data_path):
    if filename.endswith(".csv"):
        df = pd.read_csv(os.path.join(data_path, filename), 
                         sep=' ',
                         comment="#", 
                         header=None,
                         names=["timestamp", "tx", "ty", "tz", "qx", "qy", "qz", "qw"])

        slices = get_n_element_slices(df, 200)
        all_slices.extend(slices)


In [35]:
# Verify data shape
print("No. of slices: ", len(all_slices))
all_slices[0].head()
all_slices[1].head()
all_slices[-1].head()

No. of slices:  306316


,timestamp,tx,ty,tz,qx,qy,qz,qw
18500,0.000,-7.301122,5.882995,-0.172172,0.047817,-0.087781,0.311445,-0.944992
18501,0.002,-7.301709,5.876524,-0.171893,0.048026,-0.087579,0.310667,-0.945256
18502,0.004,-7.302289,5.870057,-0.171619,0.048235,-0.087374,0.309889,-0.945520
18503,0.006,-7.302861,5.863594,-0.171349,0.048445,-0.087168,0.309111,-0.945783
18504,0.008,-7.303425,5.857136,-0.171084,0.048654,-0.086959,0.308333,-0.946045


In [36]:
# Save slices
out_path = "../data/clean/slices"
for i, slice in enumerate(all_slices):
    slice.to_csv(
        os.path.join(out_path, f"slice{i}.csv")
    )